In [2]:
(define nil ())

# Map and Reduce

## Example: Reduce

`reduce` is a procedure that takes a `procedure`, a list `s`, and a `start` starting value. `reduce` combines `start` with the first element in `s` and then combine the rest of the elements using `procedure`. Thus, if we have the following,

In [ ]:
(reduce * '(3 4 5) 2)

Then the outcome should be 120. And if we have the following procedure,

In [ ]:
(reduce (lambda (x y) (cons y x)) '(3 4 5) '(2))

Then the outcome should be `(5 4 3 2)`. 

The implementation looks like the following,

In [ ]:
(define (reduce procedure s start)
  (if (null? s) ; If we run out of elements in s, then return start
      start ; start also represents the result that we have accumulated so far
      (reduce procedure (cdr s)
                        ; The whole expression below represents `start` for the recursive call
                        (procedure start (car s)) 
              ) ; End of reduce within if statement
      ) ; End of if statement
  ) ; End of reduce definition

If we analyze the procedure above,

In [ ]:
(if (null? s)
      start
      (reduce procedure (cdr s)
                        (procedure start (car s))
              ) ; End of reduce within if statement
      ) ; End of if statement

...the whole `if` statement is in a tail context. And within the `if` statement above,

In [ ]:
(reduce procedure (cdr s)
                  (procedure start (car s))
              ) ; End of reduce within if statement

This recursive `reduce` call is in a tail context. However, the 2nd argument of this `reduce` call,

In [ ]:
(procedure start (car s))

...is not in a tail context since it involves combining 2 values!

The recursive call is a tail call. However, the other calls are not. Whether `reduce` will take constant space depends on whether the argument `procedure` requires constant space. 

## Example: Map with Only a Constant Number of Frames

`Map` is a function that applies a procedure to every element in the list and constructs a list containing all the results. The non-tail-recursive implementation is as the following,

In [5]:
(define (map procedure s)
  (if (null? s)
      nil
      ; Otherwise, construct a list where the `.first` is the result of applying procedure on `car s`
      (cons (procedure (car s))
            ; and the `.rest` is a recursive call of map on the rest of the list.
            (map procedure (cdr s)))))

Now let's say we want to call the following,

In [6]:
(map (lambda (x) (- 5 x)) (list 1 2))

(4 3)

We start with a list containing `1` and `2` and bind it to `s`.

<img src = 's1.png' width = 300/>

The second call to `map` takes the rest of `s`.

<img src = 's2.png' width = 300/>

And it repeats until it reaches `nil`. The final result from returning the last call is `nil`. 

<img src = 's3.png' width = 300/>

When we call the `map` for the first time, we compute the following,

In [ ]:
(cons (- 5 1) (map procedure (cdr s)))

...and when we call `map` for the second time, we're computing the following,

In [ ]:
(cons (- 5 2) (map procedure (nil)))

And we obtain `nil` for calling `map` on the 3rd time. Working backwards for the return value, we start with returning `nil`, then from the 2nd call we obtain a list that starts with `3`.

<img src = 's4.png' width = 300/>

Then from the first call we obtain a list that starts with `4`.

<img src = 's5.png' width = 300/>

This is not a tail-recursive procedure because the following expression,

In [ ]:
(cons (procedure (car s))
            ; and the `.rest` is a recursive call of map on the rest of the list.
            (map procedure (cdr s)))

...is not in a tail context! How do we make `map` tail recursive?

First of all, we would need to implement the `reverse` procedure that reverses a list `s`. On top of that, we need to make sure that `reverse` is a tail-call procedure.

In [1]:
(define (reverse s)
  (define (reverse-iter s r) ; r is the result that we have so far, which is the reversed list
    (if (null? s)
        r
        (reverse-iter (cdr s)
                      (cons (car s) r))))
  (reverse-iter s nil))

In [3]:
(reverse '(1 2 3))

(3 2 1)

Now the tail-recursive implementation of `map` is as the following,

In [ ]:
(define (map procedure s)
  (define (map-reverse s m) ; m is the result that we have so far, but in a reversed order.
    (if (null? s)
        m
        (map-reverse (cdr s)
                     (cons (procedure (car s))
                           m))))
  (reverse (map-reverse s nil))) ; since m is in a reversed order, we need to reverse it to fix the order.

This is one of the more complex examples of making a procedure tail-recursive. 